In [1]:
import oars.pep
from oars.matrices import *
import numpy as np
np.set_printoptions(precision=3, suppress=True, linewidth=200)
from importlib import reload
# import algorithm_test


In [67]:
from oars.matrices import getMT
Z, W = getMinResist(4,  fixed_W={(3, 0): 0}, fixed_Z={(1, 0): 0}, gamma=1.5)
print(Z)
print(W)
getMetrics(Z, W)

[[ 2.     0.    -1.174 -0.826]
 [ 0.     2.    -0.826 -1.174]
 [-1.174 -0.826  2.     0.   ]
 [-0.826 -1.174  0.     2.   ]]
[[ 1.76  -0.704 -1.056  0.   ]
 [-0.704  2.6   -0.84  -1.056]
 [-1.056 -0.84   2.6   -0.704]
 [ 0.    -1.056 -0.704  1.76 ]]
No operators provided, using default LipschitzStronglyMonotoneOperator


(3.999999999837023, 0.8710384634942585, 6.0, 23.999999999022137)

In [2]:
Z, W = getTwoBlockSimilar(4)
M = getIncidence(W)
print(M)
M.T@M

[[-1.  0.  1.  0.]
 [ 0. -1.  1.  0.]
 [-1.  0.  0.  1.]
 [ 0. -1.  0.  1.]]


array([[ 2.,  0., -1., -1.],
       [ 0.,  2., -1., -1.],
       [-1., -1.,  2.,  0.],
       [-1., -1.,  0.,  2.]])

In [15]:
from oars.utils import psdCone, psdConeAlt

n = 5
dim = (n, n)
X = np.random.randn(n,n)
X = X + X.T
minmin = sum(X[X < 0])
print(minmin)
print(X)
eigs = np.linalg.eigvalsh(X)
print(eigs)
d = sum(e**2 for e in eigs if e < 0)
YB  = X + d**0.5 *np.eye(n)
print(YB)
print(np.linalg.eigvalsh(YB))
print(np.linalg.norm(YB - X, 'fro'))
print(np.linalg.norm(YB - X, 2))
YC = X - eigs[0]*np.eye(n)
print(YC)
print(np.linalg.eigvalsh(YC))
print(np.linalg.norm(YC - X, 'fro'))
print(np.linalg.norm(YC - X, 2))

-21.110182400713047
[[-1.239  0.48   0.609 -2.292 -0.995]
 [ 0.48  -1.723 -2.476  0.201  1.482]
 [ 0.609 -2.476 -1.012 -0.921  0.091]
 [-2.292  0.201 -0.921 -3.767  1.027]
 [-0.995  1.482  0.091  1.027  4.062]]
[-5.199 -4.308 -0.046  0.984  4.89 ]
[[ 5.513  0.48   0.609 -2.292 -0.995]
 [ 0.48   5.029 -2.476  0.201  1.482]
 [ 0.609 -2.476  5.74  -0.921  0.091]
 [-2.292  0.201 -0.921  2.985  1.027]
 [-0.995  1.482  0.091  1.027 10.815]]
[ 1.553  2.444  6.707  7.736 11.642]
15.098692670029207
6.752340636312622
[[ 3.96   0.48   0.609 -2.292 -0.995]
 [ 0.48   3.476 -2.476  0.201  1.482]
 [ 0.609 -2.476  4.187 -0.921  0.091]
 [-2.292  0.201 -0.921  1.432  1.027]
 [-0.995  1.482  0.091  1.027  9.262]]
[ 0.     0.891  5.153  6.183 10.089]
11.625692811205099
5.199167882277046


In [16]:
cone = psdCone(dim)
coneAlt = psdConeAlt(dim)
Y = cone.prox(X)
print(Y)
print( np.linalg.eigvalsh(Y) )
print( np.linalg.norm(Y-X, ord='fro') )
print( np.linalg.norm(Y-X, 2) )

[[ 0.236 -0.32   0.238 -0.212 -0.915]
 [-0.32   0.566 -0.563  0.307  0.949]
 [ 0.238 -0.563  0.677 -0.249 -0.39 ]
 [-0.212  0.307 -0.249  0.194  0.782]
 [-0.915  0.949 -0.39   0.782  4.201]]
[-0.     0.     0.     0.984  4.89 ]
6.7523406363126215
5.199167882277043


In [12]:
YA = coneAlt.prox(X)
print(YA)
print( np.linalg.eigvalsh(YA) )
print( np.linalg.norm(YA-X) )
print( np.linalg.norm(YA-X, ord='fro') )


[[ 7.41  -0.997 -1.176 -1.824  0.892]
 [-0.997  1.956  0.339  0.597  2.213]
 [-1.176  0.339  5.504  0.288 -2.207]
 [-1.824  0.597  0.288  3.438 -2.006]
 [ 0.892  2.213 -2.207 -2.006  5.827]]
[ 0.     2.628  4.51   6.952 10.046]
8.12901091728069
8.12901091728069


In [61]:
from scipy.linalg.lapack import dsyevr


def fastalt(X, n_sq):
    m = np.max(X)
    eigs, mat, num, isupp, info = dsyevr(X, compute_v=1, range='V', vl=0, vu=m*n_sq, abstol=1e-6)
    YB = mat[:,:num] @ np.diag(eigs[0:num]) @ mat[:,:num].T
    return YB

def fastalt2(X):
    minmin = sum(X[X < 0])
    eigs, mat, num, isupp, info = dsyevr(X, compute_v=1, range='V', vl=minmin, vu=0, abstol=1e-6)
    YB = X - mat[:,:num] @ np.diag(eigs[0:num]) @ mat[:,:num].T
    return YB

def fastalt3(X, n_sq):
    m = np.min(X)
    eigs, mat, num, isupp, info = dsyevr(X, compute_v=1, range='V', vl=m*n_sq, vu=0, abstol=1e-6)
    YB = X - mat[:,:num] @ np.diag(eigs[0:num]) @ mat[:,:num].T
    return YB

print(fastalt(X, 25))
fastalt2(X)

[[ 6.815  0.991  1.001 ... -0.675  0.453 -0.197]
 [ 0.991  6.762 -0.409 ...  0.013  0.24  -0.734]
 [ 1.001 -0.409  6.389 ... -1.31  -0.737  0.342]
 ...
 [-0.675  0.013 -1.31  ...  4.551 -0.45   0.081]
 [ 0.453  0.24  -0.737 ... -0.45   5.047 -1.448]
 [-0.197 -0.734  0.342 ...  0.081 -1.448  4.531]]


array([[ 6.815,  0.991,  1.001, ..., -0.675,  0.453, -0.197],
       [ 0.991,  6.762, -0.409, ...,  0.013,  0.24 , -0.734],
       [ 1.001, -0.409,  6.389, ..., -1.31 , -0.737,  0.342],
       ...,
       [-0.675,  0.013, -1.31 , ...,  4.551, -0.45 ,  0.081],
       [ 0.453,  0.24 , -0.737, ..., -0.45 ,  5.047, -1.448],
       [-0.197, -0.734,  0.342, ...,  0.081, -1.448,  4.531]])

In [47]:
m = np.max(X)
print(m)
n = 5
eigs, mat, num, isupp, info = dsyevr(X, compute_v=1, range='V', vl=0, vu=m*n**2, abstol=1e-6)
YB = mat[:,:num] @ np.diag(eigs[0:num]) @ mat[:,:num].T
print(YB)

2.565681985594567
[[ 0.678 -0.552 -0.144 -1.315  0.057]
 [-0.552  0.484 -0.06   1.175  0.154]
 [-0.144 -0.06   0.944 -0.252 -1.046]
 [-1.315  1.175 -0.252  2.862  0.49 ]
 [ 0.057  0.154 -1.046  0.49   1.174]]


In [52]:
X = np.random.randn(n,n)

In [62]:
np.random.seed(1)
n = 100
n_sq = n**2
dim = (n, n)
for i in range(5):
    print(i)
    X = np.random.randn(n,n)
    X = X + X.T
    cone = psdCone(dim)
    
    %timeit cone.prox(X)
    %timeit fastalt(X, n_sq)
    %timeit fastalt2(X)
    %timeit fastalt3(X, n_sq)

0
759 µs ± 109 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
5.64 ms ± 641 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.4 ms ± 730 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.69 ms ± 298 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1
1.04 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
7.93 ms ± 545 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.28 ms ± 195 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.69 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2
988 µs ± 44.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
7.71 ms ± 260 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.57 ms ± 252 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.78 ms ± 202 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3
1.02 ms ± 76.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
7.77 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loo

In [54]:
%timeit fastalt(X)

19.1 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [55]:
%timeit fastalt2(X)

22.3 µs ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [57]:
%timeit fastalt3(X)

20 µs ± 2.97 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [63]:
tgt_n = 60
Z, W = getFull(tgt_n)
Ko, K1, Ki, Kp = oars.pep.getConstraintMatrices(Z, W, gamma=0.5)

G = sum([-Ko, K1, (1+1/tgt_n)*Ki] + Kp)
print(G)

%timeit [np.trace(A @ G) for A in Kp]

# Make sparse
sparseKp = [sparse.csr_matrix(A) for A in Kp]
sparseG = sparse.csr_matrix(G)

%timeit [(A.multiply(sparseG)).sum() for A in sparseKp]


[[ 0.017  1.     1.    ... -0.017 -0.017 -0.017]
 [ 1.     0.017  1.    ... -0.017 -0.017 -0.017]
 [ 1.     1.     0.017 ... -0.017 -0.017 -0.017]
 ...
 [-0.017 -0.017 -0.017 ... -0.019  0.067  0.068]
 [-0.017 -0.017 -0.017 ...  0.067 -0.018  0.068]
 [-0.017 -0.017 -0.017 ...  0.068  0.068 -0.017]]
7.14 ms ± 936 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
18.9 ms ± 5.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
n = 6
Z, W = getTwoBlockSimilar(n)
print(Z)
print(W)

[[ 2.     0.     0.    -0.667 -0.667 -0.667]
 [ 0.     2.     0.    -0.667 -0.667 -0.667]
 [ 0.     0.     2.    -0.667 -0.667 -0.667]
 [-0.667 -0.667 -0.667  2.     0.     0.   ]
 [-0.667 -0.667 -0.667  0.     2.     0.   ]
 [-0.667 -0.667 -0.667  0.     0.     2.   ]]
[[ 2.     0.     0.    -0.667 -0.667 -0.667]
 [ 0.     2.     0.    -0.667 -0.667 -0.667]
 [ 0.     0.     2.    -0.667 -0.667 -0.667]
 [-0.667 -0.667 -0.667  2.     0.     0.   ]
 [-0.667 -0.667 -0.667  0.     2.     0.   ]
 [-0.667 -0.667 -0.667  0.     0.     2.   ]]


In [7]:
n = 6
Z, W = getBlockMin(n, n//2, objective=getMaxConnectivity)
print(Z)
print(W)
print(np.linalg.eigvalsh(Z))
print(np.linalg.eigvalsh(W))

[[ 2.     0.     0.    -0.667 -0.667 -0.667]
 [ 0.     2.    -0.    -0.667 -0.667 -0.667]
 [ 0.    -0.     2.    -0.667 -0.667 -0.667]
 [-0.667 -0.667 -0.667  2.    -0.    -0.   ]
 [-0.667 -0.667 -0.667 -0.     2.    -0.   ]
 [-0.667 -0.667 -0.667 -0.    -0.     2.   ]]
[[ 1.756 -0.244 -0.244 -0.423 -0.423 -0.423]
 [-0.244  1.756 -0.244 -0.423 -0.423 -0.423]
 [-0.244 -0.244  1.756 -0.423 -0.423 -0.423]
 [-0.423 -0.423 -0.423  1.756 -0.244 -0.244]
 [-0.423 -0.423 -0.423 -0.244  1.756 -0.244]
 [-0.423 -0.423 -0.423 -0.244 -0.244  1.756]]
[0. 2. 2. 2. 2. 4.]
[-0.     2.     2.     2.     2.     2.538]


In [5]:
n = 8
Z, W = getBlockMin(n, n//2, objective=getMinSLEM)
print(Z)
print(W)
print(np.linalg.eigvalsh(Z))
print(np.linalg.eigvalsh(W))

[[ 2.  -0.  -0.  -0.  -0.5 -0.5 -0.5 -0.5]
 [-0.   2.   0.   0.  -0.5 -0.5 -0.5 -0.5]
 [-0.   0.   2.   0.  -0.5 -0.5 -0.5 -0.5]
 [-0.   0.   0.   2.  -0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5  2.   0.   0.   0. ]
 [-0.5 -0.5 -0.5 -0.5  0.   2.   0.   0. ]
 [-0.5 -0.5 -0.5 -0.5  0.   0.   2.   0. ]
 [-0.5 -0.5 -0.5 -0.5  0.   0.   0.   2. ]]
[[ 1.75 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25]
 [-0.25  1.75 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25]
 [-0.25 -0.25  1.75 -0.25 -0.25 -0.25 -0.25 -0.25]
 [-0.25 -0.25 -0.25  1.75 -0.25 -0.25 -0.25 -0.25]
 [-0.25 -0.25 -0.25 -0.25  1.75 -0.25 -0.25 -0.25]
 [-0.25 -0.25 -0.25 -0.25 -0.25  1.75 -0.25 -0.25]
 [-0.25 -0.25 -0.25 -0.25 -0.25 -0.25  1.75 -0.25]
 [-0.25 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25  1.75]]
[0. 2. 2. 2. 2. 2. 2. 4.]
[0. 2. 2. 2. 2. 2. 2. 2.]


In [7]:
import oars.matrices.prebuilt

Z, W = oars.matrices.prebuilt.getTwoBlockSLEM(n)
print(Z)
print(W)

[[ 2.   0.   0.   0.  -0.5 -0.5 -0.5 -0.5]
 [ 0.   2.   0.   0.  -0.5 -0.5 -0.5 -0.5]
 [ 0.   0.   2.   0.  -0.5 -0.5 -0.5 -0.5]
 [ 0.   0.   0.   2.  -0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5  2.   0.   0.   0. ]
 [-0.5 -0.5 -0.5 -0.5  0.   2.   0.   0. ]
 [-0.5 -0.5 -0.5 -0.5  0.   0.   2.   0. ]
 [-0.5 -0.5 -0.5 -0.5  0.   0.   0.   2. ]]
[[ 1.75 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25]
 [-0.25  1.75 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25]
 [-0.25 -0.25  1.75 -0.25 -0.25 -0.25 -0.25 -0.25]
 [-0.25 -0.25 -0.25  1.75 -0.25 -0.25 -0.25 -0.25]
 [-0.25 -0.25 -0.25 -0.25  1.75 -0.25 -0.25 -0.25]
 [-0.25 -0.25 -0.25 -0.25 -0.25  1.75 -0.25 -0.25]
 [-0.25 -0.25 -0.25 -0.25 -0.25 -0.25  1.75 -0.25]
 [-0.25 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25  1.75]]


In [45]:
for i in range(Z.shape[0]):
    for j in range(Z.shape[1]):
        if abs(Z[i,j]) < 1e-5:
            Z[i,j] = 0

Z[0].nonzero()


(array([0, 4, 5, 6, 7], dtype=int64),)

In [35]:
Z, W = getBlockMin(n, n//2, objective=getMinResist)
print(Z)
print(W)
print(np.linalg.eigvalsh(Z))
print(np.linalg.eigvalsh(W))

[[ 2.   0.   0.   0.  -0.5 -0.5 -0.5 -0.5]
 [ 0.   2.   0.   0.  -0.5 -0.5 -0.5 -0.5]
 [ 0.   0.   2.   0.  -0.5 -0.5 -0.5 -0.5]
 [ 0.   0.   0.   2.  -0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5  2.   0.   0.   0. ]
 [-0.5 -0.5 -0.5 -0.5  0.   2.   0.   0. ]
 [-0.5 -0.5 -0.5 -0.5  0.   0.   2.   0. ]
 [-0.5 -0.5 -0.5 -0.5  0.   0.   0.   2. ]]
[[ 2.  -0.  -0.  -0.  -0.5 -0.5 -0.5 -0.5]
 [-0.   2.  -0.  -0.  -0.5 -0.5 -0.5 -0.5]
 [-0.  -0.   2.  -0.  -0.5 -0.5 -0.5 -0.5]
 [-0.  -0.  -0.   2.  -0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5  2.  -0.  -0.  -0. ]
 [-0.5 -0.5 -0.5 -0.5 -0.   2.  -0.  -0. ]
 [-0.5 -0.5 -0.5 -0.5 -0.  -0.   2.  -0. ]
 [-0.5 -0.5 -0.5 -0.5 -0.  -0.  -0.   2. ]]
[0. 2. 2. 2. 2. 2. 2. 4.]
[-0.  2.  2.  2.  2.  2.  2.  4.]


In [37]:
Zsparse = sparse.csr_array()
Zsparse.prune()
print(Zsparse.indices)
print(Zsparse.data)
print(Zsparse.indptr)
Zsparse.toarray()
abs(Z)>1e-5


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
[ 2.  -0.5 -0.5 -0.5 -0.5  2.  -0.5 -0.5 -0.5 -0.5  2.  -0.5 -0.5 -0.5 -0.5  2.  -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5  2.  -0.5 -0.5 -0.5 -0.5  2.  -0.5 -0.5 -0.5 -0.5  2.  -0.5 -0.5 -0.5 -0.5
  2. ]
[ 0 40]


array([[ True, False, False, False,  True,  True,  True,  True],
       [False,  True, False, False,  True,  True,  True,  True],
       [False, False,  True, False,  True,  True,  True,  True],
       [False, False, False,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True, False, False, False],
       [ True,  True,  True,  True, False,  True, False, False],
       [ True,  True,  True,  True, False, False,  True, False],
       [ True,  True,  True,  True, False, False, False,  True]])

In [53]:
Zsparse = sparse.csr_matrix(Z)
from scipy.sparse.linalg import eigsh

print(eigsh(Zsparse))
print(np.linalg.eigvalsh(Z))

(array([2., 2., 2., 2., 2., 4.]), array([[-0.003,  0.   , -0.003,  0.   ,  0.   , -0.354],
       [ 0.01 ,  0.039,  0.001, -0.81 ,  0.095, -0.354],
       [-0.075, -0.016, -0.001,  0.322, -0.746, -0.354],
       [ 0.067, -0.023,  0.003,  0.488,  0.651, -0.354],
       [ 0.234, -0.243, -0.797, -0.011, -0.022,  0.354],
       [ 0.399, -0.497,  0.584, -0.024, -0.042,  0.354],
       [-0.853, -0.087,  0.087, -0.004,  0.086,  0.354],
       [ 0.22 ,  0.827,  0.126,  0.04 , -0.023,  0.354]]))
[-0.  2.  2.  2.  2.  2.  2.  4.]


In [52]:
Zsparse.shape

(8, 8)

In [11]:
Dimension = 2**12
from scipy.sparse import rand
A = rand(Dimension, Dimension, density=0.001, format='csr')
B = rand(Dimension, Dimension, density=0.001, format='csr')

(A.T * B).trace()

3.3498984029572885

In [19]:
from scipy import sparse

dim = (4,4)
M = sparse.csr_array(dim, dtype=np.float64)
M += sparse.eye(dim[0], dim[1], format='csr')
M *= 2
M.toarray()

array([[2., 0., 0., 0.],
       [0., 2., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 2.]])

In [16]:
%timeit (A.multiply(B)).sum()

406 µs ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
%timeit sum((A.T @ B).diagonal())

1.33 ms ± 70.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
import cvxpy as cvx
from oars.matrices import getCore
Z, W, cons = getCore(4, fixed_W={(3, 0): 0})
obj = cvx.Minimize(cvx.norm(Z-W, 'fro'))
prob = cvx.Problem(obj, cons)
prob.solve()
print(Z.value)
print(W.value)

var34300
var34299
[[ 2. -1. -1. -0.]
 [-1.  2. -0. -1.]
 [-1. -0.  2. -1.]
 [-0. -1. -1.  2.]]
[[ 2. -1. -1.  0.]
 [-1.  2.  0. -1.]
 [-1.  0.  2. -1.]
 [ 0. -1. -1.  2.]]


In [17]:
W

array([[ 2.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167,
        -0.167],
       [-0.   ,  2.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167,
        -0.167],
       [-0.   , -0.   ,  2.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167,
        -0.167],
       [-0.   , -0.   , -0.   ,  2.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167,
        -0.167],
       [-0.   , -0.   , -0.   , -0.   ,  2.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.167, -0.1

In [2]:
from oars.utils import *
from oars.matrices import *
import numpy as np
t = [1, 2, 3]
l = np.array([[0, 1, 2], [1,0,1], [2,1,0]])
Z, W = getMT(3)
fig = getGantt(t, l, Z, W)
fig.show()

In [9]:
# getMetrics Example
from oars.utils import getMetrics
from oars.matrices import getBlockMin
n = 4
Z, W = getBlockMin(n, n//2)
t = np.array([1, 2, 3, 4])
l = np.array([[0,1,2,3], 
              [1,0,1,2], 
              [2,1,0,1], 
              [3,2,1,0]])
ls = np.array([5, 4, 3, 2])
mus = np.array([3, 2, 1, 0])
contraction_target = 0.0001
cycle, tau, total_cycles, total_time = getMetrics(Z, W, t, l, ls, mus, contraction_target)
print('Cycle:', cycle)
print('Tau:', tau)
print('Total Cycles:', total_cycles)
print('Total Time:', total_time)


No operators provided, using default LipschitzStronglyMonotoneOperator
Cycle: 10.999999999999147
Tau: 0.8083945725558891
Total Cycles: 44.0
Total Time: 483.9999999999625


In [11]:
np.set_printoptions(precision=3, suppress=True, linewidth=200)
from oars.matrices import getMinCycle, getMaxConnectivity
Zmc, Wmc = getMinCycle(n, objective=getMaxConnectivity, t=t, l=l, minW=n//2)
print(Zmc)
print(Wmc)
cycle, tau, total_cycles, total_time = getMetrics(Zmc, Wmc, t, l, ls, mus, contraction_target)
print('Cycle:', cycle)
print('Tau:', tau)
print('Total Cycles:', total_cycles)
print('Total Time:', total_time)

[[ 2. -1. -1. -0.]
 [-1.  2. -0. -1.]
 [-1. -0.  2. -1.]
 [-0. -1. -1.  2.]]
[[ 2. -1. -1.  0.]
 [-1.  2.  0. -1.]
 [-1.  0.  2. -1.]
 [ 0. -1. -1.  2.]]
No operators provided, using default LipschitzStronglyMonotoneOperator
Cycle: 9.99999999999865
Tau: 0.8208356029559916
Total Cycles: 47.0
Total Time: 469.99999999993656


In [1]:
reload(algorithm_test)

NameError: name 'reload' is not defined

In [2]:
algorithm_test.testSDP(parallel=True)


In [54]:
from oars.matrices import getFull, getMT
from oars.pep import getConstraintMatrices
import proxs
reload(proxs)
from oars import solveMT
n = 3
Z, W = getMT(n)
Ko, K1, Ki, Kp = getConstraintMatrices(Z, W, gamma=0.5)
u = np.ones(2*n)
u[n:] = 0
# proxlist = [proxs.traceHalfspaceIndicator, proxs.traceEqualityIndicator, proxs.traceHalfspaceIndicator,]# for _ in Kp]
# proxlist = []
proxlist = [proxs.psdCone, proxs.traceEqualityIndicator, proxs.traceEqualityIndicator, proxs.linearSubdiff] + [proxs.traceHalfspaceIndicator for _ in Kp] #, linearSubdiff]
data = [(2*n, 2*n), {'A':Ki, 'v':1}, {'A':K1, 'v':0}, -Ko] + Kp #, u, {'A':Ki, 'v':1}]#, -Ko]
#data = [Kp[0], {'A':Ki, 'v':1}, Kp[1]]# + [u, (2*n, 2*n)]#, -Ko]
dim = len(data)
Wd, Zd = getBlockMin(dim, dim//2)
from oars import solve
x, results = solve(dim, data, proxlist, W=Wd, Z=Zd, itrs=100000, vartol=1e-4, gamma=0.8, checkperiod=10, verbose=False)
# x, results = solveMT(dim, data, proxlist, itrs=10000, vartol=1e-6, gamma=0.8, checkperiod=10, verbose=False)
oneprox = proxs.traceEqualityIndicator({'A':Ki, 'v':1})
print(oneprox.prox(x))
print(x)
# print(x)
# print(results)
# print(np.trace(Ko @ x))


Converged in objective value, iteration 35356
[[ 0.502 -0.022 -0.474  0.143  0.095 -0.053]
 [-0.022  0.029 -0.004  0.023  0.005  0.02 ]
 [-0.474 -0.004  0.469 -0.161 -0.098  0.034]
 [ 0.143  0.023 -0.161  0.072  0.037  0.003]
 [ 0.095  0.005 -0.098  0.037  0.021 -0.004]
 [-0.053  0.02   0.034  0.003 -0.004  0.017]]
[[ 0.502 -0.022 -0.474  0.143  0.095 -0.053]
 [-0.022  0.029 -0.004  0.023  0.005  0.02 ]
 [-0.474 -0.004  0.469 -0.161 -0.098  0.034]
 [ 0.143  0.023 -0.161  0.072  0.037  0.003]
 [ 0.095  0.005 -0.098  0.037  0.021 -0.004]
 [-0.053  0.02   0.034  0.003 -0.004  0.017]]


In [55]:
np.trace(Ko @ x)

0.8806363180682683

In [53]:
from oars.pep import getContractionFactor
print(getContractionFactor(Z, W, verbose=True))

optimal
0.8772093960780886
[[ 0.497 -0.024 -0.473  0.142  0.092 -0.056]
 [-0.024  0.027 -0.003  0.021  0.004  0.019]
 [-0.473 -0.003  0.476 -0.163 -0.097  0.037]
 [ 0.142  0.021 -0.163  0.071  0.036  0.002]
 [ 0.092  0.004 -0.097  0.036  0.02  -0.005]
 [-0.056  0.019  0.037  0.002 -0.005  0.017]]
1.057491839126227
0.24253000172637015
1.8391259270014304
0.2831436413187557
2.0488014245113737
0.365486443518793
0.877208753564782
804.0745487528972
0.8772093960780886


In [49]:
for Kpi in Kp:
    print(np.trace(Kpi @ x))
    half = proxs.traceHalfspaceIndicator(Kpi)
    y = half.prox(x)
    print(np.trace(Kpi @ y))
    print(np.linalg.eigvalsh(Kpi))

cone = proxs.reducedCone(u)
yy = cone.prox(x)
print(yy)
print(np.linalg.eigvalsh(yy))
cone.getEigen(x)
print(np.linalg.eigvalsh(x))
# print(cone.prox(x))

-1.726780265748351e-05
0.0
[-2.118  0.     0.    -0.    -0.     0.118]
-5.286842805712899e-05
-5.551115123125783e-17
[-1.236 -0.     0.     0.     0.     3.236]
-1.3989199108756234e-05
0.0
[-2.225 -0.     0.     0.    -0.     0.225]
3.3899369720605255e-05
3.3899369720605255e-05
[-2.372 -0.    -0.     0.     0.     3.372]
-0.00018755979739369655
3.469446951953614e-18
[-2.323 -0.     0.     0.     0.     0.323]
-0.0003738813230230731
1.3877787807814457e-17
[-3.464 -0.    -0.    -0.     0.     3.464]
[[ 0.508 -0.021 -0.478  0.146  0.101 -0.044]
 [-0.021  0.03  -0.012  0.028  0.007  0.022]
 [-0.478 -0.012  0.451 -0.158 -0.101  0.027]
 [ 0.146  0.028 -0.158  0.073  0.039  0.007]
 [ 0.101  0.007 -0.101  0.039  0.023 -0.002]
 [-0.044  0.022  0.027  0.007 -0.002  0.016]]
[-0.012 -0.     0.     0.     0.081  1.032]
[-0.001 -0.    -0.     0.     0.082  1.032]


In [25]:
# [[ 0.786  0.521  0.521  0.142  0.     0.   ]
#  [ 0.521  0.857  0.502 -0.041  0.     0.   ]
#  [ 0.521  0.502  0.857 -0.041  0.     0.   ]
#  [ 0.142 -0.041 -0.041  0.08   0.     0.   ]
#  [ 0.     0.     0.     0.     0.     0.   ]
#  [ 0.     0.     0.     0.     0.     0.   ]]

xx = np.array([[ 0.786,  0.521,  0.521,  0.142,  0.   ,  0.   ],
    [ 0.521,  0.857,  0.502, -0.041,  0.   ,  0.   ],
        [ 0.521,  0.502,  0.857, -0.041,  0.   ,  0.   ],
        [ 0.142, -0.041, -0.041,  0.08 ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

np.linalg.eig(xx)

EigResult(eigenvalues=array([ 1.864,  0.362, -0.   ,  0.355,  0.   ,  0.   ]), eigenvectors=array([[-0.566, -0.723, -0.396, -0.   ,  0.   ,  0.   ],
       [-0.583,  0.358,  0.178, -0.707,  0.   ,  0.   ],
       [-0.583,  0.358,  0.178,  0.707,  0.   ,  0.   ],
       [-0.018, -0.469,  0.883, -0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ]]))

In [21]:
traceeq = traceEqualityIndicator({'A':Ki, 'v':1})
print(traceeq.U)

[[0.333 0.    0.    0.    0.    0.   ]
 [0.    0.333 0.    0.    0.    0.   ]
 [0.    0.    0.333 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]]


In [15]:
W@W

array([[ 1.  ,  0.  ,  0.  , -1.  ,  0.5 ,  0.5 ],
       [ 0.  ,  1.  ,  0.  ,  0.5 , -1.  ,  0.5 ],
       [ 0.  ,  0.  ,  1.  ,  0.5 ,  0.5 , -1.  ],
       [-1.  ,  0.5 ,  0.5 ,  1.5 , -0.75, -0.75],
       [ 0.5 , -1.  ,  0.5 , -0.75,  1.5 , -0.75],
       [ 0.5 ,  0.5 , -1.  , -0.75, -0.75,  1.5 ]])

In [16]:
W@W

array([[ 6., -3., -3.],
       [-3.,  6., -3.],
       [-3., -3.,  6.]])

In [19]:
y = oneprox.prox(x)
np.trace(Ki @ y)
Ki

array([[1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [9]:
import proxs
import timeit
X = np.random.randn(10, 10)

# Make symmetric A
A = X + X.T

aprox = proxs.traceHalfspaceIndicator(A)
A.shape[0]
# print(aprox.scale(A))
# print(aprox.other_scale(A))
# np.allclose(aprox.scale(A), aprox.other_scale(A))
# # Compare the times for scale and otherscale
print(timeit.timeit('aprox.scale(A)', number=1000, globals=globals()))

# timeit.timeit('aprox.other_scale(A)', number=1000, globals=globals())
# q = 0
# for i in range(A.shape[0]):
#     for j in range(i):
#         q += 2*A[i,j]**2
#     q += A[i,i]**2

# print(q)
# print(np.linalg.norm(A, 'fro')**2)

0.006233800027985126


In [13]:
reload(proxs)

<module 'proxs' from 'c:\\Users\\peter\\OneDrive\\Documents\\code\\oars\\tests\\proxs.py'>

In [13]:
Y = 2*A + np.eye(A.shape[0])
np.trace(A@Y)

350.94063175498553

In [14]:
aprox = proxs.traceEqualityIndicator({'A':A, 'v':1})
Yp = aprox.prox(A, 1)
print(np.trace(A@Yp))

1.0000000000000016


In [5]:
np.linalg.norm(A, 'fro')**2

255.65986269415086

In [2]:
n=4
Z, W = getMT(n)
L = - np.tril(Z, -1)
ls = np.ones(n)*2
ls = np.cumsum(ls)
mus = np.ones(n)
mus = np.cumsum(mus)
operators = [oars.pep.SmoothStronglyConvexFunction(L=ls[i], mu=mus[i]) for i in range(n)]
tau = oars.pep.getContractionFactor(Z, W, None, None, operators=operators, alpha=1, gamma=0.5)
W, tau_opt = oars.pep.getOptimalW(Z, operators=operators)
Wo, tauo = oars.pep.getSubdiffW(L, ls=ls, mus=mus)
print(W)
print(tau_opt)
print(tau)
print(Wo)
print(tauo)

[[ 2.43588727 -1.09730184 -0.25396727 -1.08461816]
 [-1.09730184  3.237486   -1.4169987  -0.72318545]
 [-0.25396727 -1.4169987   3.79612127 -2.1251553 ]
 [-1.08461816 -0.72318545 -2.1251553   3.93295892]]
0.14316698043646195
0.8993338570188733
[[ 2.43588727 -1.09730184 -0.25396727 -1.08461816]
 [-1.09730184  3.237486   -1.4169987  -0.72318545]
 [-0.25396727 -1.4169987   3.79612127 -2.1251553 ]
 [-1.08461816 -0.72318545 -2.1251553   3.93295892]]
0.14316698043646195


In [2]:
n=4
Z, W = getMT(n)
L = - np.tril(Z, -1)
ls = np.ones(n)*2
ls = np.cumsum(ls)
mus = np.ones(n)
mus = np.cumsum(mus)
operators = [oars.pep.LipschitzStronglyMonotoneOperator(L=ls[i], mu=mus[i]) for i in range(n)]
tau = oars.pep.getContractionFactor(Z, W, None, None, operators=operators, alpha=1, gamma=0.5)
W, tau_opt = oars.pep.getOptimalW(Z, operators=operators)
Wo, tauo = oars.pep.getSubdiffW(L, ls=ls, mus=mus)
print(W)
print(tau_opt)
print(tau)
print(Wo)
print(tauo)

[[ 2.10883221 -1.011507   -0.3138347  -0.7834905 ]
 [-1.011507    2.97677056 -1.18071865 -0.7845449 ]
 [-0.3138347  -1.18071865  3.00502589 -1.51047254]
 [-0.7834905  -0.7845449  -1.51047254  3.07850794]]
0.6562344018316276
0.9924137484022097
[[ 2.43588727 -1.09730184 -0.25396727 -1.08461816]
 [-1.09730184  3.237486   -1.4169987  -0.72318545]
 [-0.25396727 -1.4169987   3.79612127 -2.1251553 ]
 [-1.08461816 -0.72318545 -2.1251553   3.93295892]]
0.14316698043646195


In [2]:
n = 4
Z, W = getMT(n)
L = -np.tril(Z, -1)
ls = np.ones(n)*2
mus = np.ones(n)
Wopt, tau = oars.pep.getOptimalW(Z, ls=ls, mus=mus)
print(Wopt)
print(tau)

[[ 6.98255568e-09 -6.92177773e-09  3.66397885e-11  3.66397881e-11]
 [-6.92177773e-09  7.39331880e-09 -1.95265932e-10 -1.95265931e-10]
 [ 3.66397885e-11 -1.95265932e-10  1.85725841e-11  1.85725869e-11]
 [ 3.66397881e-11 -1.95265931e-10  1.85725869e-11  1.85725855e-11]]
1.0000000006201917


In [4]:
reload(oars.pep)

<module 'oars.pep' from 'C:\\Users\\peter\\OneDrive\\Documents\\code\\oars\\oars\\pep\\__init__.py'>

In [4]:
n = 3
Z, W = getMT(n)
M = getIncidence(W)

Ko, K1, Ki, Kp = getConstraintMatrices(Z, W)
Ko

array([[ 1.,  0.,  0., -1.,  1., -0.],
       [ 0.,  1.,  0.,  1., -2.,  1.],
       [ 0.,  0.,  1., -0.,  1., -1.],
       [-1.,  1., -0.,  2., -3.,  1.],
       [ 1., -2.,  1., -3.,  6., -3.],
       [-0.,  1., -1.,  1., -3.,  2.]])

In [5]:
K1

array([[1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [6]:
Ki

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [8]:
n = 3
Z, W = getMT(n)
eye = np.eye(n)
ones = np.ones((n,n))
zeros = np.zeros((n,n))
Ko, K1, Ki, Kp = getConstraintMatrices(Z, W)
Ko_ref = np.block([[eye, -W], [-W, W@W]])
assert(np.allclose(Ko, Ko_ref))

In [10]:
K1_ref = np.block([[ones, zeros], [zeros, zeros]])
assert(np.allclose(K1, K1_ref))
Ki_ref = np.block([[eye, zeros], [zeros, zeros]])
assert(np.allclose(Ki, Ki_ref))

In [13]:
for k in Kp:
    print(str(k))

[[-1. -0. -0.  1.  0.  0.]
 [-0. -0. -0.  0.  0.  0.]
 [-0. -0. -0.  0.  0.  0.]
 [ 1.  0.  0.  3.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[ 0.   0.   0.   0.5  0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.5  0.   0.  -2.  -0.  -0. ]
 [ 0.   0.   0.  -0.  -0.  -0. ]
 [ 0.   0.   0.  -0.  -0.  -0. ]]
[[-0. -0. -0. -0.  0.  0.]
 [-0. -1. -0. -1.  1.  0.]
 [-0. -0. -0. -0.  0.  0.]
 [-0. -1. -0. -1.  1.  0.]
 [ 0.  1.  0.  1.  3.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.5  0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.5  0. ]
 [ 0.   0.5  0.   0.5 -2.  -0. ]
 [ 0.   0.   0.   0.  -0.  -0. ]]
[[-0. -0. -0. -0. -0.  0.]
 [-0. -0. -0. -0. -0.  0.]
 [-0. -0. -1. -1. -1.  1.]
 [-0. -0. -1. -1. -1.  1.]
 [-0. -0. -1. -1. -1.  1.]
 [ 0.  0.  1.  1.  1.  3.]]
[[ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.5]
 [ 0.   0.   0

In [14]:
Kp

[array([[-1., -0., -0.,  1.,  0.,  0.],
        [-0., -0., -0.,  0.,  0.,  0.],
        [-0., -0., -0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  3.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]]),
 array([[ 0. ,  0. ,  0. ,  0.5,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0.5,  0. ,  0. , -2. , -0. , -0. ],
        [ 0. ,  0. ,  0. , -0. , -0. , -0. ],
        [ 0. ,  0. ,  0. , -0. , -0. , -0. ]]),
 array([[-0., -0., -0., -0.,  0.,  0.],
        [-0., -1., -0., -1.,  1.,  0.],
        [-0., -0., -0., -0.,  0.,  0.],
        [-0., -1., -0., -1.,  1.,  0.],
        [ 0.,  1.,  0.,  1.,  3.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]]),
 array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0.5,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0.5,  0. ],
        [ 0. ,  0.5,  0. ,  0.5, -2. , -0. ],
        

In [15]:
n = 3
Z, W = getMT(n)
M = getIncidence(W)
Ko, Ki, Kmus, Kls = getGramsOld(Z, M)
Kon, Kin, Kpn = getReducedConstraintMatrices(Z, M)

No operators provided, using default LipschitzStronglyMonotoneOperator


In [16]:
Kon

array([[ 1.,  0., -1.,  1.,  0.],
       [ 0.,  1.,  0., -1.,  1.],
       [-1.,  0.,  1., -1.,  0.],
       [ 1., -1., -1.,  2., -1.],
       [ 0.,  1.,  0., -1.,  1.]])

In [19]:
n = 3
Z, W = getMT(n)
M = getIncidence(W)
d = M.shape[0]
Ko, Ki, Kmus, Kls = getGramsOld(Z, M)
Kon, Kin, Kpn = getReducedConstraintMatrices(Z, M)
eye = np.eye(d)
Ko_ref = np.block([[eye, M], [M.T, M.T@M]])
assert(np.allclose(Ko_ref, Kon))

No operators provided, using default LipschitzStronglyMonotoneOperator


In [20]:
Ki

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [22]:

    Ki_ref = np.zeros((d+n, d+n))
Ki_ref[:d, :d] = eye
assert(np.allclose(Ki_ref, Kin))

In [23]:
Kpn

[array([[-1.,  0.,  1.,  0.,  0.],
        [ 0., -0., -0., -0., -0.],
        [ 1., -0.,  3.,  0.,  0.],
        [ 0., -0.,  0.,  0.,  0.],
        [ 0., -0.,  0.,  0.,  0.]]),
 array([[ 0. ,  0. ,  0.5,  0. ,  0. ],
        [ 0. ,  0. , -0. , -0. , -0. ],
        [ 0.5, -0. , -2. , -0. , -0. ],
        [ 0. , -0. , -0. , -0. , -0. ],
        [ 0. , -0. , -0. , -0. , -0. ]]),
 array([[-1.,  1.,  1., -1., -0.],
        [ 1., -1., -1.,  1.,  0.],
        [ 1., -1., -1.,  1.,  0.],
        [-1.,  1.,  1.,  3.,  0.],
        [-0.,  0.,  0.,  0.,  0.]]),
 array([[ 0. ,  0. , -0. , -0.5, -0. ],
        [ 0. ,  0. ,  0. ,  0.5,  0. ],
        [-0. ,  0. ,  0. ,  0.5,  0. ],
        [-0.5,  0.5,  0.5, -2. , -0. ],
        [-0. ,  0. ,  0. , -0. , -0. ]]),
 array([[-0., -0.,  0.,  0., -0.],
        [-0., -1.,  1.,  1., -1.],
        [ 0.,  1., -1., -1.,  1.],
        [ 0.,  1., -1., -1.,  1.],
        [-0., -1.,  1.,  1.,  3.]]),
 array([[ 0. ,  0. , -0. , -0. , -0. ],
        [ 0. ,  0. , -0. 

In [24]:
Kpn_ref = [np.array([[-1.,  0.,  1.,  0.,  0.],
                [ 0., -0., -0., -0., -0.],
                [ 1., -0.,  3.,  0.,  0.],
                [ 0., -0.,  0.,  0.,  0.],
                [ 0., -0.,  0.,  0.,  0.]]),
        np.array([[ 0. ,  0. ,  0.5,  0. ,  0. ],
                [ 0. ,  0. , -0. , -0. , -0. ],
                [ 0.5, -0. , -2. , -0. , -0. ],
                [ 0. , -0. , -0. , -0. , -0. ],
                [ 0. , -0. , -0. , -0. , -0. ]]),
        np.array([[-1.,  1.,  1., -1., -0.],
                [ 1., -1., -1.,  1.,  0.],
                [ 1., -1., -1.,  1.,  0.],
                [-1.,  1.,  1.,  3.,  0.],
                [-0.,  0.,  0.,  0.,  0.]]),
        np.array([[ 0. ,  0. , -0. , -0.5, -0. ],
                [ 0. ,  0. ,  0. ,  0.5,  0. ],
                [-0. ,  0. ,  0. ,  0.5,  0. ],
                [-0.5,  0.5,  0.5, -2. , -0. ],
                [-0. ,  0. ,  0. , -0. , -0. ]]),
        np.array([[-0., -0.,  0.,  0., -0.],
                [-0., -1.,  1.,  1., -1.],
                [ 0.,  1., -1., -1.,  1.],
                [ 0.,  1., -1., -1.,  1.],
                [-0., -1.,  1.,  1.,  3.]]),
        np.array([[ 0. ,  0. , -0. , -0. , -0. ],
                [ 0. ,  0. , -0. , -0. , -0.5],
                [-0. , -0. ,  0. ,  0. ,  0.5],
                [-0. , -0. ,  0. ,  0. ,  0.5],
                [-0. , -0.5,  0.5,  0.5, -2. ]])]
for i in range(2*n):
    assert(np.allclose(Kpn[i], Kpn_ref[i]))